In [1]:
pip install git+https://github.com/PrithivirajDamodaran/Styleformer.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Styleformer.git to /tmp/pip-req-build-a7q4fw3j
  Running command git clone -q https://github.com/PrithivirajDamodaran/Styleformer.git /tmp/pip-req-build-a7q4fw3j
     |████████████████████████████████| 5.5 MB 3.9 MB/s 
     |████████████████████████████████| 1.3 MB 65.1 MB/s 
     |████████████████████████████████| 175 kB 58.1 MB/s 
     |████████████████████████████████| 2.2 MB 20.0 MB/s 
     |████████████████████████████████| 7.6 MB 64.8 MB/s 
     |████████████████████████████████| 182 kB 93.3 MB/s 
  Created wheel for styleformer: filename=styleformer-0.1-py3-none-any.whl size=8624 sha256=ccfe22d8e6fa7b617972365e18a1fa0dba35a9837d2baaaee9a0b868e9ae7bdb
  Stored in directory: /tmp/pip-ephem-wheel-cache-tam67mpf/wheels/85/3e/e9/5f30f549b352fb24bdc7287b0bf59f78de70af14090d336be0
Successfully built styleformer


In [2]:
from styleformer import Styleformer
import torch
import warnings
warnings.filterwarnings("ignore")

In [3]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [4]:
'''
#uncomment for re-producability
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(1234)
'''

# style = [0=Casual to Formal, 1=Formal to Casual, 2=Active to Passive, 3=Passive to Active etc..]
sf = Styleformer(style = 0) 


source_sentences = ["But I'm thrilled to be here with thousands of hardworking patriots as we celebrate the miracle of Christmas, the greatness of America and the glory of God."]

for source_sentence in source_sentences:
    target_sentence = sf.transfer(source_sentence)
    print("-" *100)
    print("[Casual] ", source_sentence)
    print("-" *100)
    if target_sentence is not None:
        print("[Formal] ",target_sentence)
        print()
    else:
        print("No good quality transfers available !")

Downloading:   0%|          | 0.00/913 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Casual to Formal model loaded...
----------------------------------------------------------------------------------------------------
[Casual]  But I'm thrilled to be here with thousands of hardworking patriots as we celebrate the miracle of Christmas, the greatness of America and the glory of God.
----------------------------------------------------------------------------------------------------
[Formal]  I am thrilled to be here, together with thousands of hardworking patriots, as we celebrate the miracle of Christmas, the greatness of America,



In [6]:
from nltk.translate.bleu_score import sentence_bleu
from nltk import ngrams

In [50]:
def PINCscore(s, c, N): # s:source sentence. c:target side paraphrase. N: maximum n-gram
  s = s.lower().split()
  c = c.lower().split()
  terminate = 0
  if len(c) < N:
    N = len(c)
  for n in range(N):
    ngram_s = set(ngrams(s, n+1))
    ngram_c = set(ngrams(c, n+1))
    terminate += len(ngram_s.intersection(ngram_c))/len(ngram_c)
  score = 1-1.0/N*terminate
  return round(score, 4)
  

In [37]:
source_sentences = ['The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, & the Democrats know it', 'Almost all recent elections using this system, even though much smaller &  with far fewer Ballots to count, have ended up being a disaster.', 'Large numbers of missing Ballots & Fraud!']
epoch = 10
target_sentences = []
num = 0
for source_sentence in source_sentences:
  print("-" *100)
  print("[Casual] ", source_sentence)
  print("-" *100)
  for i in range(10):
    #target_sentence = sf.transfer(source_sentence, inference_on=0, quality_filter=0.5, max_candidates=5)
    target_sentence = sf.transfer(source_sentence)
    if target_sentence is not None:
      blue_score = round(sentence_bleu([target_sentence], source_sentence), 4)
      pinc_score = PINCscore(source_sentence, target_sentence, 4)
      if blue_score < 0.9 and pinc_score > 0.5:
        print("[Formal] ",target_sentence)
        print(blue_score, pinc_score)
        target_sentences.append([target_sentence, blue_score, pinc_score])
        break
  num+=1
  print(len(target_sentences), num)
  if len(target_sentences) < num:
    print("None")
    target_sentences.append([])
    num+=1
    

----------------------------------------------------------------------------------------------------
[Casual]  The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, & the Democrats know it
----------------------------------------------------------------------------------------------------
0 1
None
----------------------------------------------------------------------------------------------------
[Casual]  Almost all recent elections using this system, even though much smaller &  with far fewer Ballots to count, have ended up being a disaster.
----------------------------------------------------------------------------------------------------
[Formal]  Nearly all recent elections using this system have ended up being a disaster even though they were much smaller and had far fewer ballots.
0.7805 0.598
2 3
None
----------------------------------------------------------------------------------------------------
[Casual]  Large numbers of missing Ballots & Fraud!
-------

In [40]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
source_sentences = []
f = open('BattleCreekDec19_2019.txt', 'r', encoding="utf-8")
file_input = f.readlines()
for cnt, line in enumerate(file_input):
    sentences = nltk.sent_tokenize(line)
    # print(sentences)
    for i , sent in enumerate(sentences):
        source_sentences.append(sent)

In [42]:
len(source_sentences)

1893

In [59]:
epoch = 10
target_sentences = []
num = 0

for source_sentence in source_sentences[130:150]:
  isok = False
  print("-" *100)
  print("[Casual] ", source_sentence)
  print("-" *100)
  for i in range(10):
    #target_sentence = sf.transfer(source_sentence, inference_on=0, quality_filter=0.5, max_candidates=5)
    target_sentence = sf.transfer(source_sentence)
    if target_sentence is not None:
      blue_score = round(sentence_bleu([target_sentence], source_sentence), 4)
      pinc_score = PINCscore(source_sentence, target_sentence, 4)
      if blue_score < 0.9 and pinc_score > 0.5:
        print("[Formal] ",target_sentence)
        print(blue_score, pinc_score)
        target_sentences.append([target_sentence, blue_score, pinc_score])
        isok = True
        break
  num+=1
  print(len(target_sentences), num)
  if not isok:
    print("None")
    target_sentences.append([source_sentence, 1, 0])

----------------------------------------------------------------------------------------------------
[Casual]  What happens is all of you people go crazy.
----------------------------------------------------------------------------------------------------
[Formal]  What happens is that all of you people get crazy.
0.7994 0.5316
1 1
----------------------------------------------------------------------------------------------------
[Casual]  Look, look, look, and the place.
----------------------------------------------------------------------------------------------------
[Formal]  Look at the place.
0.4065 0.7917
2 2
----------------------------------------------------------------------------------------------------
[Casual]  Okay, so there's one disgusting person who made… Wait, wait, who made a… I wouldn't say this, but made a horrible gesture with the wrong finger right?
----------------------------------------------------------------------------------------------------
[Formal]  O

In [60]:
print(" ".join(source_sentences[130:150]))

What happens is all of you people go crazy. Look, look, look, and the place. Okay, so there's one disgusting person who made… Wait, wait, who made a… I wouldn't say this, but made a horrible gesture with the wrong finger right? Now they won't say that, the fake news, they won't say it. If one of us did that, it would be like the biggest story ever. I think it's terrible. And I'll tell you the other thing, I don't know who the security company is, but the police came up, but they want to be so politically correct so they don't grab her wrist lightly and get her out. They say, "Oh, would you please come? Would you please come with me?" Sure. "Ma'am, would you?" And then she gives the guy the finger and he, "Oh, Oh." You got to get a little bit stronger than that, folks. And they… . Making another group of tariffs, seven and a half percent. This gives us a lot of currency to do the phase two deal, which they want to start, but we're taking in billions of dollars and to help the farmers wh

In [66]:
# print average scores
bleu = 0
pinc = 0
for sentence in target_sentences:
  bleu+=sentence[1]
  pinc+=sentence[2]
avg_bleu = bleu/len(target_sentences)
avg_pinc = pinc/len(target_sentences)

In [67]:
print("avg BLEU:", avg_bleu)
print("avg PINC:", avg_pinc)

avg BLEU: 0.6435449999999998
avg PINC: 0.57026


In [62]:
output_sentences = []
for sentence in target_sentences:
  output_sentences.append(sentence[0])
print(" ".join(output_sentences))

What happens is that all of you people get crazy. Look at the place. Okay, so there is one disgusting person who made a terrible gesture with the wrong finger, correct? Now they will not say that fake news. If one of us did that, it would be like the biggest story ever. I think it is terrible. In addition, I'm not aware of the security company, but the police have come up, which tries to keep things politically correct so they don They say Oh, would you please come? Would you please be willing to accompany me? Sure. I ask if you would like to do this, Mother. Then she gave him the finger, and he said Oh. People, you need to a bit more strong then that. And they. Making another group of tariffs, seven and a half percent. This provides us with some currency to initiate the Phase Two agreement. However, we are taking in billions of dollars to help the farmers who were targeted. Well, what we do out of that large check that we receive, all of that money flows into the Treasury. We gave the

In [64]:
# save output
phras = " ".join(output_sentences)
with open("output.txt", "w") as f:
  f.write(phras)